In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-08-13"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
48,2024-08-13,República Dominicana Lnb,20:30,Indios,Reales de la Vega,1.66,2.03,177.5,1.83,1.83,0.0,0.0,0.0,EwiSfMv9,0.602410,0.492611,0.546448,0.546448,0.095020,151.404,15.876695,0.104863,1.2,1.643168,1.369306,178.48,1.748,0.287002,0.164189,-6.0,138.438,20.555584,0.148482,2.4,1.341641,0.559017,138.32,1.574,0.205134,0.130327,23.0,92,91,1.94,1.52,127.806,0.0,0.141805,0.000000,NaN,0.79,0.158,4.177215,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
49,2024-08-13,República Dominicana Lnb,21:00,Caneros,Marineros,1.40,2.66,172.5,1.88,1.78,0.0,0.0,0.0,zguyg09L,0.714286,0.375940,0.531915,0.561798,0.090226,175.496,30.602489,0.174377,3.0,0.000000,0.000000,173.00,2.070,0.474710,0.229329,28.0,183.178,40.463816,0.220899,0.0,0.000000,NaN,196.56,2.196,0.448141,0.204071,-51.0,100,84,1.73,2.34,0.000,0.0,0.438894,0.038640,NaN,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
50,2024-08-13,República Dominicana Lnb,21:30,Metros de Santiago,Titanes Del Licey,1.46,2.47,174.5,1.80,1.86,0.0,0.0,0.0,Wxt6mvXr,0.684932,0.404858,0.555556,0.537634,0.089790,155.538,40.153023,0.258156,1.2,1.643168,1.369306,129.00,1.754,0.360943,0.205783,-7.0,170.222,40.055575,0.235314,1.2,1.643168,1.369306,204.70,1.904,0.358511,0.188293,9.0,86,89,1.50,2.30,0.000,0.0,0.363449,0.023184,NaN,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
51,2024-08-13,Vietnã Vba,09:30,Da Nang Dragons,Saigon Heat,4.20,1.17,162.5,1.82,1.84,0.0,0.0,0.0,tr8nKZgE,0.238095,0.854701,0.549451,0.543478,0.092796,408.178,331.134624,0.811251,0.6,1.341641,2.236068,254.81,5.000,3.667486,0.733497,-17.0,110.996,20.441294,0.184162,1.8,1.643168,0.912871,144.32,1.406,0.160873,0.114419,30.0,83,88,3.07,1.64,0.000,0.0,0.797964,0.007728,NaN,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0


## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
